<a href="https://colab.research.google.com/github/pkonowrocki/Multiple-pixels-attack/blob/master/MultiplePixelsAttack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
file_id = '1Iz4CTYuTT-_eS1HrIU410VkeomP5dl0R'

#authentication
from google.colab import auth
auth.authenticate_user()

#build Gdrive service
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

try:
  import io
  from googleapiclient.http import MediaIoBaseDownload

  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    _, done = downloader.next_chunk()
  downloaded.seek(0)
  print('Downloaded file')
  import pickle
  dataset = pickle.load(downloaded)
  assert type(dataset)!="<class 'dict'>", "Something went wrong with downloading file"
  print("Dataset ready")

except:
  print("Something wrong with file")
  print("Trying to download the file and upload it to a Gdrive")
  #downloading dataset
  !curl http://ml.cs.tsinghua.edu.cn/~chenxi/dataset/val224_compressed.pkl --output val224_compressed.pkl
    
  #mounting Gdrive 
  from google.colab import drive
  drive.mount('/content/gdrive')
  #upload dataset to Gdrive
  from googleapiclient.http import MediaFileUpload
  file_metadata = {
    'name': 'val224_compressed'
  }
  media = MediaFileUpload('val224_compressed.pkl',
                        resumable=True)
  created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
  print('File ID: {}'.format(created.get('id')))

In [0]:
#install required files
!pip3 install torch torchvision

In [48]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print(device)

import torchvision.models as models
net = models.alexnet(pretrained=True).to(device)
net.eval()

print("Network ready")

cuda:0


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth
100%|██████████| 244418560/244418560 [00:02<00:00, 95502587.41it/s]


Network ready


In [49]:
#find positive examples
import numpy as np
import cv2
import torch
import torch.nn

def str2img(str_b):
    return cv2.imdecode(np.fromstring(str_b, np.uint8), cv2.IMREAD_COLOR)


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

print("Got "+str(len(dataset["target"])) +" samples")
m = nn.Softmax()
positive = []

for i in range(len(dataset["target"])):
  image = str2img(dataset["data"][i])
  image = torch.from_numpy(image).view(1, 3, 224, 224).to(device=device, dtype=torch.float)
  image.div_(255.)
  image[:, 0, :, :].sub_(mean[0]).div_(std[0])
  image[:, 1, :, :].sub_(mean[1]).div_(std[1])
  image[:, 2, :, :].sub_(mean[2]).div_(std[2])
  label = dataset["target"][i]
  output = net(image)
  output = m(output)
  values, indices = torch.max(output, 1)
  if(label==indices.item()):
    print("Image #"+str(i)+"\tprobability: "+str(values.item()*100))
    positive.append((i, values.item()*100))

print(positive)

Got 50000 samples


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Image #125	probability: 70.38662433624268
Image #230	probability: 29.68287467956543
Image #1121	probability: 34.66236591339111
Image #1204	probability: 80.14419674873352
Image #1293	probability: 99.26235675811768
Image #1734	probability: 22.51041829586029
Image #4460	probability: 30.719149112701416
Image #4914	probability: 22.840631008148193
Image #5251	probability: 31.67714774608612
Image #5763	probability: 10.272357612848282
Image #6234	probability: 20.63409835100174
Image #6729	probability: 77.3688018321991
Image #6906	probability: 24.49241131544113
Image #7640	probability: 20.38220465183258
Image #8581	probability: 88.46515417098999
Image #9537	probability: 35.77677011489868
Image #9614	probability: 59.678393602371216
Image #10247	probability: 50.593119859695435
Image #11610	probability: 35.52859425544739
Image #11749	probability: 75.22026896476746
Image #12299	probability: 57.825249433517456
Image #12488	probability: 98.24746251106262
Image #13385	probability: 26.576286554336548
I

In [50]:
positive = [(125, 70.38662433624268), (230, 29.68287467956543), (1121, 34.66236591339111), (1204, 80.14419674873352), (1293, 99.26235675811768), (1734, 22.51041829586029), (4460, 30.719149112701416), (4914, 22.840631008148193), (5251, 31.67714774608612), (5763, 10.272357612848282), (6234, 20.63409835100174), (6729, 77.3688018321991), (6906, 24.49241131544113), (7640, 20.38220465183258), (8581, 88.46515417098999), (9537, 35.77677011489868), (9614, 59.678393602371216), (10247, 50.593119859695435), (11610, 35.52859425544739), (11749, 75.22026896476746), (12299, 57.825249433517456), (12488, 98.24746251106262), (13385, 26.576286554336548), (13627, 41.33410155773163), (13661, 21.732445061206818), (13734, 39.57911729812622), (14359, 14.517764747142792), (15384, 83.67582559585571), (15639, 11.083117872476578), (18307, 8.994264155626297), (19241, 68.18674802780151), (19474, 89.1351580619812), (19753, 72.94465899467468), (20060, 69.57603096961975), (20698, 78.03838849067688), (21335, 56.468266248703), (21461, 77.93958783149719), (22785, 65.87823033332825), (23355, 35.305508971214294), (25701, 29.582718014717102), (26628, 43.83417367935181), (26703, 8.497815579175949), (26822, 37.64067590236664), (27731, 29.25107181072235), (29598, 98.43541383743286), (29876, 66.34024977684021), (31332, 34.71480906009674), (31924, 33.64042639732361), (31969, 76.82187557220459), (32663, 48.51921200752258), (33350, 98.14919829368591), (35491, 67.31398105621338), (39049, 64.29259181022644), (39705, 86.5737795829773), (40235, 35.74826121330261), (40244, 48.99843633174896), (42084, 53.020232915878296), (42275, 13.818688690662384), (42591, 52.02655792236328), (42972, 44.17412579059601), (43095, 71.31219506263733), (44495, 6.340372562408447), (45326, 23.5866978764534), (46550, 72.07267880439758), (47259, 26.392918825149536), (48606, 85.98803877830505), (48816, 49.71528351306915), (48953, 52.26907134056091), (49117, 18.790780007839203), (49648, 34.377798438072205), (49709, 56.61540627479553)]

71


Sources: [Dataset](https://github.com/aaron-xichen/pytorch-playground) [Neural network model](https://pytorch.org/docs/stable/torchvision/models.html)